In [1]:

import requests
import pymysql  
import datetime
import time


API_KEY = "QnZOc0T8rMyDtdLY7SxsqWJ4LI3fh484XtMfgPhz"

url =  f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"


response = requests.get(url)


In [2]:
target = 10000 # Total close approach records
asteroids_data = []
approach_data = []

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"


while len(approach_data) < target:
    response = requests.get(url)
    data = response.json()
    neo_data = data.get("near_earth_objects", {})

    for date, asteroid_list in neo_data.items():
        for asteroid in asteroid_list:
            try:
               
                asteroids_data.append({
                    'id': asteroid.get('id'),
                    'name': asteroid.get('name'),
                    'absolute_magnitude_h': float(asteroid.get('absolute_magnitude_h', 0)),
                    'estimated_diameter_min_km': asteroid.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0),
                    'estimated_diameter_max_km': asteroid.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0),
                    'is_potentially_hazardous_asteroid': asteroid.get('is_potentially_hazardous_asteroid', False)
                })

                
                for approach in asteroid.get('close_approach_data', []):
                    try:
                        approach_data.append({
                            'neo_reference_id': asteroid.get('id'),
                            'close_approach_date': datetime.datetime.strptime(approach.get('close_approach_date'), "%Y-%m-%d").date(),
                            'relative_velocity_kmph': float(approach.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
                            'astronomical': float(approach.get('miss_distance', {}).get('astronomical', 0.0)),
                            'miss_distance_km': float(approach.get('miss_distance', {}).get('kilometers', 0.0)),
                            'miss_distance_lunar': float(approach.get('miss_distance', {}).get('lunar', 0.0)),
                            'orbiting_body': approach.get('orbiting_body', 'Unknown')
                        })
                    except Exception as e:
                        print(f"Skipped approach record due to error: {e}")

                    if len(approach_data) >= target:
                        break
                if len(approach_data) >= target:
                    break

            except Exception as e:
                print(f"Skipped asteroid record due to error: {e}")
        if len(approach_data) >= target:
            break

    url = data['links'].get('next')
    if not url:
        break
    time.sleep(1)

    

print(f" Fetched {len(asteroids_data)} asteroid records.")
print(f" Fetched {len(approach_data)} close approach records.")

 Fetched 10000 asteroid records.
 Fetched 10000 close approach records.


In [3]:
import pymysql


connection = pymysql.connect(
    host = "localhost",
    user = "root",
    password = "rakshitha@2906",
    database = "asteroids"
)

cursor = connection.cursor()

In [4]:

cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id VARCHAR(255),
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")


cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id VARCHAR(255),
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(255)
)
""")


0

In [5]:

asteroid_insert_query = """
INSERT INTO asteroids (
    id, name, absolute_magnitude_h, 
    estimated_diameter_min_km, estimated_diameter_max_km, 
    is_potentially_hazardous_asteroid
) VALUES (%s, %s, %s, %s, %s, %s)
"""

for a in asteroids_data:
    cursor.execute(asteroid_insert_query, (
        a['id'],
        a['name'],
        a['absolute_magnitude_h'],
        a['estimated_diameter_min_km'],
        a['estimated_diameter_max_km'],
        a['is_potentially_hazardous_asteroid']
    ))


approach_insert_query = """
INSERT INTO close_approach (
    neo_reference_id, close_approach_date, relative_velocity_kmph,
    astronomical, miss_distance_km, miss_distance_lunar, orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for ca in approach_data:
    cursor.execute(approach_insert_query, (
        ca['neo_reference_id'],
        ca['close_approach_date'],
        ca['relative_velocity_kmph'],
        ca['astronomical'],
        ca['miss_distance_km'],
        ca['miss_distance_lunar'],
        ca['orbiting_body']
    ))


connection.commit()
cursor.close()
connection.close()

print(" Data successfully inserted into MySQL.")

 Data successfully inserted into MySQL.
